In [20]:
%reset -f

In [1]:
import pandas as pd

from tqdm import tqdm
tqdm.pandas()

import gspread as gs
import pandas as pd

from datetime import datetime

In [2]:
import google.auth
from google.oauth2 import service_account
from google.cloud import bigquery

credentials = service_account.Credentials.from_service_account_file('Cashbac-GCP-Keys.json')
project_id = 'cashbac-31433'

# Make clients.
bqclient = bigquery.Client(
    credentials=credentials,
    project=project_id,
)

In [3]:
import sys
import os

cur_directory=os.getcwd()
parent_directory=cur_directory[:cur_directory.rfind('/')]
sys.path.append(parent_directory)
 
# importing
import MyFunc
from MyFunc import *

from importlib import reload
reload(MyFunc)
from MyFunc import *

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [4]:
query_string = '''

SELECT
google_id,place_id,
working_hours__Monday Monday
,working_hours__Tuesday Tuesday
,working_hours__Wednesday Wednesday
,working_hours__Thursday Thursday
,working_hours__Friday Friday
,working_hours__Saturday Saturday
,working_hours__Sunday Sunday
from cashbac-31433.cashbac_datalake_prod.vw_outcrapper_google_results_raw table1
where true
and table1.working_hours__Monday is not null
and not EXISTS 
(
  select 1 
  from cashbac_datalake_prod.outcrapper_google_merchants_business_time table2
  where table1.google_id=table2.google_id
)

'''

query_result = list(bqclient.query(query_string).result(timeout=None))
business_time = pd.DataFrame(data=[list(x.values()) for x in tqdm(query_result)], columns=list(query_result[0].keys()))
business_time=business_time.drop_duplicates()
del query_result
print(len(business_time))
business_time.head()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 85/85 [00:00<00:00, 199952.80it/s]

85


,google_id,place_id,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,0x2e69f1b8e6e9439b:0x8778f96231cd97c9,ChIJm0Pp5rjxaS4RyZfNMWL5eIc,"""10AM-9:30PM""","""10AM-9:30PM""","""10AM-9:30PM""","""10AM-9:30PM""","""10AM-9:30PM""","""10AM-9:30PM""","""10AM-9:30PM"""
1,0x2e69f630607ae9ef:0x289bdc766cb79230,ChIJ7-l6YDD2aS4RMJK3bHbcmyg,"""10AM-9:30PM""","""10AM-9:30PM""","""10AM-9:30PM""","""10AM-9:30PM""","""10AM-9:30PM""","""10AM-9:30PM""","""10AM-9:30PM"""
2,0x2e69f5346b73474f:0x6840024e01765f29,ChIJT0dzazT1aS4RKV92AU4CQGg,"""10AM-10PM""","""10AM-10PM""","""10AM-10PM""","""10AM-10PM""","""10AM-10PM""","""10AM-10PM""","""10AM-10PM"""
3,0x2e68e11c4a9d27e5:0x7bcd1ace08104f55,ChIJ5SedShzhaC4RVU8QCM4azXs,"""9AM-6PM""","""9AM-6PM""","""9AM-6PM""","""9AM-6PM""","""9AM-6PM""","""8AM-7PM""","""8AM-7PM"""
4,0x2e68e10296a13433:0x8f9e8f11076eff0b,ChIJMzShlgLhaC4RC_9uBxGPno8,"""Closed""","""9AM-5PM""","""9AM-8PM""","""9AM-8PM""","""9AM-8PM""","""Closed""","""9AM-8PM"""


In [5]:
business_time.columns[-7:]
def get_business_time(opening_text):
    opening_text=str(opening_text).replace('"','')
    opening_text=opening_text.encode('ascii', 'ignore').decode("utf-8")
    result_open=[]
    result_closed=[]
    if opening_text=='':
        pass
    elif opening_text=='Closed':
        result_open.append('Closed')
        result_closed.append('Closed')
    elif opening_text=='Open 24 hours':
        result_open.append('00:00')
        result_closed.append('24:00')
    else:
        opens_time=opening_text.split(',')
        for time in opens_time:
            try:
                open=time.split('-')[0]
                closed=time.split('-')[1]
                if (closed[-2:]=='AM' or closed[-2:]=='PM') and open[-2:]!='AM' and open[-2:]!='PM':
                    open=open+closed[-2:]
                if open.count(':')>0:
                    open=datetime.strptime(open, '%I:%M%p').strftime('%H:%M')
                else:
                    open=datetime.strptime(open, '%I%p').strftime('%H:%M')

                if closed.count(':')>0:
                    closed=datetime.strptime(closed, '%I:%M%p').strftime('%H:%M')
                else:
                    closed=datetime.strptime(closed, '%I%p').strftime('%H:%M')

                result_open.append(open)
                result_closed.append(closed)
            except:
                print(opening_text)

    return result_open,result_closed

In [6]:
# result_table=pd.DataFrame(columns=['google_id','place_id','day_week','start_hour','end_hour'])
# result_table.to_csv('results/outcrapper_google_merchants_business_time.csv',sep=';',index=False)

In [7]:
pd.DataFrame(columns=['google_id','place_id','day_week','start_hour','end_hour'])\
    .to_csv('results/outcrapper_google_merchants_business_time.csv',sep=';',index=False)

In [8]:
daysname=business_time.columns[-7:]
for i in tqdm(range(len(business_time))):
    start_hour=None
    end_hour=None
    temp_data=business_time.iloc[i]
    for day in daysname:
        start_hour,end_hour=get_business_time(temp_data[day])
        for open,close in zip(start_hour,end_hour):
            # result_table.loc[len(result_table)]=[temp_data.google_id,temp_data.place_id,day,open,close]

            pd.DataFrame(columns=['google_id','place_id','day_week','start_hour','end_hour']
                         ,data=[[temp_data.google_id,temp_data.place_id,day,open,close]]
                        ).to_csv('results/outcrapper_google_merchants_business_time.csv',sep=';',index=False,mode='a',header=False)
            

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 85/85 [00:00<00:00, 463.65it/s]


In [9]:
# %%time
# result_table.to_csv('results/outcrapper_google_merchants_business_time.csv',sep=';',index=False)

In [10]:
import sys
import os

cur_directory=os.getcwd()
parent_directory=cur_directory[:cur_directory.rfind('/')]
sys.path.append(parent_directory)
 
# importing
import MyFunc
from MyFunc import *

from importlib import reload
reload(MyFunc)
from MyFunc import *

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [11]:
%%time
from google.cloud import bigquery

df_temp=pd.read_csv('results/outcrapper_google_merchants_business_time.csv',sep=';', nrows=1)

my_schema=[]
for column in df_temp.columns :
    my_schema.append(bigquery.SchemaField(column, "STRING"))
    
cashbac_key='Cashbac-GCP-Keys.json'
horego_key='Horego-GCP-Keys.json'


# src_path_file='results/outcrapper_google_all_filtered.csv'
# target_path_file='scrapping_results/outcrapper_google_all_filtered.csv'
# bucket_name='horego-bq'
# upload_file_to_GCS(horego_key,bucket_name,src_path_file,target_path_file)

# table_schema = "horego_datalake_dev"
# table_name = "outcrapper_google_results"
# gcs_path_uri = "gs://horego-bq/scrapping_results/outcrapper_google_all_filtered.csv"
# create_table_from_gcs(horego_key,gcs_path_uri,table_schema,table_name,my_schema)

################################################################################
print('')

src_path_file='results/outcrapper_google_merchants_business_time.csv'
target_path_file='scrapping_results/outcrapper_google_merchants_business_time.csv'
bucket_name='cashbac_datalake'
upload_file_to_GCS(cashbac_key,bucket_name,src_path_file,target_path_file)

table_schema = "cashbac_datalake_prod"
table_name = "outcrapper_google_merchants_business_time"
gcs_path_uri = f"gs://{bucket_name}/scrapping_results/outcrapper_google_merchants_business_time.csv"
create_table_from_gcs(cashbac_key,gcs_path_uri,table_schema,table_name,my_schema,'append')

# table_schema = "public_datalake"
# table_name = "outcrapper_google_merchants_business_time"
# gcs_path_uri = f"gs://{bucket_name}/scrapping_results/outcrapper_google_merchants_business_time.csv"
# create_table_from_gcs(cashbac_key,gcs_path_uri,table_schema,table_name,my_schema,'append')



Uploading results/outcrapper_google_merchants_business_time.csv
Path in GCS: cashbac_datalake/scrapping_results/outcrapper_google_merchants_business_time.csv
table on : cashbac_datalake_prod.outcrapper_google_merchants_business_time
CPU times: user 189 ms, sys: 9.03 ms, total: 198 ms
Wall time: 4.7 s
